<a href="https://colab.research.google.com/github/ricardoruedas/ML/blob/main/%5B05%5D%20-%20Arboles%20de%20decision/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Arboles de decisión: XGBoost(Extreme Gradient Boosting) - Ejercicio 3: XGBoost.ipynb

Este notebook es un **I do**: todo resuelto y explicado paso a paso.

## Objetivos

- Creo un dataset alearotio con 5000 ejemplos para predecir si un cliente comprará.
- Entrena un XGBoost.
- Evalúa el modelo con métricas de clasificación (accuracy, matriz de confusión y reporte).
- Muestra la importancia de cada característica (qué variables usa más el modelo para decidir).

## 1) Instalamos y cargamos librerias xgboost

In [1]:
# ¡IMPORTANTE! Instalar XGBoost primero:
!pip install xgboost scikit-learn pandas numpy matplotlib

In [3]:

import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.datasets import make_classification, make_regression
from sklearn.metrics import accuracy_score, mean_squared_error, classification_report
import matplotlib.pyplot as plt
import xgboost as xgb

## 2) Vamos a crear el dataset

In [4]:
# Dataset de clasificación (predecir si un cliente comprará)
np.random.seed(42)
X_clf, y_clf = make_classification(
    n_samples=5000,  # Dataset más grande para ver diferencias
    n_features=20,
    n_informative=15,
    n_redundant=3,
    random_state=42
)

# Simulamos datos faltantes (ventaja para XGBoost!)
# Creamos índices aleatorios para filas y columnas
n_missing = int(0.1 * X_clf.shape[0] * X_clf.shape[1])  # 10% de valores faltantes
missing_rows = np.random.choice(X_clf.shape[0], size=n_missing, replace=True)
missing_cols = np.random.choice(X_clf.shape[1], size=n_missing, replace=True)
X_clf[missing_rows, missing_cols] = np.nan

print(f"📊 Dataset creado:")
print(f"   📈 Muestras: {X_clf.shape[0]:,}")
print(f"   📋 Características: {X_clf.shape[1]}")
print(f"   🕳️  Valores faltantes: {np.isnan(X_clf).sum():,}")


📊 Dataset creado:
   📈 Muestras: 5,000
   📋 Características: 20
   🕳️  Valores faltantes: 9,490


## 3) Dividimos train y test

In [5]:
# Dividir datos
X_train, X_test, y_train, y_test = train_test_split(
    X_clf, y_clf, test_size=0.3, random_state=42
)

In [6]:
# Manejo de datos faltantes

print("\n Manejo de datos faltantes")
print("-" * 50)

print("🥊 Gradient Boosting clásico:")
print("   ❌ No puede manejar NaN directamente")
print("   🔧 Necesita preprocesamiento...")

# Para GBM clásico, rellenamos NaN
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
X_train_filled = imputer.fit_transform(X_train)
X_test_filled = imputer.transform(X_test)

print("   ✅ Datos rellenados con la media")

print("\n⚡ XGBoost:")
print("   ✅ Maneja NaN automáticamente")
print("   🎯 Aprende la mejor estrategia para cada NaN")


 Manejo de datos faltantes
--------------------------------------------------
🥊 Gradient Boosting clásico:
   ❌ No puede manejar NaN directamente
   🔧 Necesita preprocesamiento...
   ✅ Datos rellenados con la media

⚡ XGBoost:
   ✅ Maneja NaN automáticamente
   🎯 Aprende la mejor estrategia para cada NaN


## 4) Entrenamos los modelos y comparamos

In [7]:
print("Velocidad de entrenamiento")
print("-" * 50)

# Configuración similar para comparación justa
n_estimators = 100
max_depth = 6
learning_rate = 0.1

print("⏱️  Midiendo tiempos de entrenamiento...")

# Gradient Boosting clásico
print("\n🐢 Entrenando Gradient Boosting clásico...")
start_time = time.time()
gbm_model = GradientBoostingClassifier(
    n_estimators=n_estimators,
    max_depth=max_depth,
    learning_rate=learning_rate,
    random_state=42
)
gbm_model.fit(X_train_filled, y_train)
gbm_time = time.time() - start_time

# XGBoost
print("🚀 Entrenando XGBoost...")
start_time = time.time()
xgb_model = xgb.XGBClassifier(
    n_estimators=n_estimators,
    max_depth=max_depth,
    learning_rate=learning_rate,
    random_state=42,
    eval_metric='logloss'  # Evita warnings
)
xgb_model.fit(X_train, y_train)
xgb_time = time.time() - start_time

print(f"\n📊 RESULTADOS DE VELOCIDAD:")
print(f"   🐢 Gradient Boosting: {gbm_time:.2f} segundos")
print(f"   🚀 XGBoost:          {xgb_time:.2f} segundos")
print(f"   ⚡ XGBoost es {gbm_time/xgb_time:.1f}x MÁS RÁPIDO!")

Velocidad de entrenamiento
--------------------------------------------------
⏱️  Midiendo tiempos de entrenamiento...

🐢 Entrenando Gradient Boosting clásico...
🚀 Entrenando XGBoost...

📊 RESULTADOS DE VELOCIDAD:
   🐢 Gradient Boosting: 8.09 segundos
   🚀 XGBoost:          1.04 segundos
   ⚡ XGBoost es 7.8x MÁS RÁPIDO!


## 5) Evaluamos los resultados de los modelos tras el entrenamiento

In [8]:
# Precisión y rendimiento
print("Precisión y rendimiento")
print("-" * 50)

# Predicciones
gbm_pred = gbm_model.predict(X_test_filled)
xgb_pred = xgb_model.predict(X_test)

# Métricas
gbm_accuracy = accuracy_score(y_test, gbm_pred)
xgb_accuracy = accuracy_score(y_test, xgb_pred)

print("🎯 PRECISIÓN:")
print(f"   🐢 Gradient Boosting: {gbm_accuracy:.4f} ({gbm_accuracy*100:.2f}%)")
print(f"   🚀 XGBoost:          {xgb_accuracy:.4f} ({xgb_accuracy*100:.2f}%)")

if xgb_accuracy > gbm_accuracy:
    diff = (xgb_accuracy - gbm_accuracy) * 100
    print(f"   🏆 XGBoost gana por +{diff:.2f} puntos porcentuales!")
else:
    print("   🤝 ¡Empate técnico!")

# Cross-validation para ser más justos
print("\n🔄 Cross-Validation (5-fold):")
gbm_cv_scores = cross_val_score(gbm_model, X_train_filled, y_train, cv=5)
xgb_cv_scores = cross_val_score(xgb_model, X_train, y_train, cv=5)

print(f"   🐢 GBM CV:  {gbm_cv_scores.mean():.4f} (±{gbm_cv_scores.std()*2:.4f})")
print(f"   🚀 XGB CV:  {xgb_cv_scores.mean():.4f} (±{xgb_cv_scores.std()*2:.4f})")


Precisión y rendimiento
--------------------------------------------------
🎯 PRECISIÓN:
   🐢 Gradient Boosting: 0.9153 (91.53%)
   🚀 XGBoost:          0.9147 (91.47%)
   🤝 ¡Empate técnico!

🔄 Cross-Validation (5-fold):
   🐢 GBM CV:  0.9009 (±0.0378)
   🚀 XGB CV:  0.8997 (±0.0309)


## 6) Vamos a interpretar los modelos

In [9]:
# Interpretabilidad
print("Interpretabilidad")
print("-" * 50)
print("📊 IMPORTANCIA DE CARACTERÍSTICAS:")

# Gradient Boosting
gbm_importance = gbm_model.feature_importances_
top_gbm_features = np.argsort(gbm_importance)[-5:][::-1]

print("\n🐢 Top 5 características - Gradient Boosting:")
for i, idx in enumerate(top_gbm_features):
    print(f"   {i+1}. Feature_{idx:2d}: {gbm_importance[idx]:.4f}")

# XGBoost (múltiples tipos de importance)
xgb_importance_gain = xgb_model.feature_importances_
top_xgb_features = np.argsort(xgb_importance_gain)[-5:][::-1]

print("\n🚀 Top 5 características - XGBoost (gain):")
for i, idx in enumerate(top_xgb_features):
    print(f"   {i+1}. Feature_{idx:2d}: {xgb_importance_gain[idx]:.4f}")

# XGBoost tiene más tipos de importance
print("\n🎯 XGBoost BONUS - Múltiples métricas de importancia:")
try:
    # Importancia por frecuencia de uso
    xgb_freq_importance = xgb_model.get_booster().get_score(importance_type='frequency')
    print(f"   📊 Por frecuencia disponible: {len(xgb_freq_importance)} features")

    # Importancia por ganancia
    xgb_gain_importance = xgb_model.get_booster().get_score(importance_type='gain')
    print(f"   🎯 Por ganancia disponible: {len(xgb_gain_importance)} features")
except:
    print("   ℹ️  Otras métricas disponibles con get_booster().get_score()")

Interpretabilidad
--------------------------------------------------
📊 IMPORTANCIA DE CARACTERÍSTICAS:

🐢 Top 5 características - Gradient Boosting:
   1. Feature_ 9: 0.1706
   2. Feature_ 2: 0.0865
   3. Feature_18: 0.0805
   4. Feature_17: 0.0751
   5. Feature_ 8: 0.0700

🚀 Top 5 características - XGBoost (gain):
   1. Feature_ 9: 0.1488
   2. Feature_ 2: 0.1047
   3. Feature_18: 0.0700
   4. Feature_17: 0.0613
   5. Feature_15: 0.0607

🎯 XGBoost BONUS - Múltiples métricas de importancia:
   ℹ️  Otras métricas disponibles con get_booster().get_score()


## 7) Un poco sobre las caracteristicas especiales y detalles de los modelos

In [10]:
# Caracteristicas Avanzadas

print("Caracteristicas Avanzadas")
print("-" * 50)

print("🔧 CARACTERÍSTICAS ESPECIALES:")

print("\n🐢 Gradient Boosting clásico:")
print("   ✅ Fácil de entender")
print("   ✅ Implementación estable")
print("   ❌ Pocos parámetros avanzados")
print("   ❌ No maneja NaN")
print("   ❌ No paralelización eficiente")

print("\n🚀 XGBoost:")
print("   ✅ Manejo automático de NaN")
print("   ✅ Regularización L1 y L2")
print("   ✅ Early stopping inteligente")
print("   ✅ Paralelización y GPU")
print("   ✅ Multiple importance metrics")
print("   ✅ Cross-validation integrada")
print("   ❌ Más parámetros para tunear")
print("   ❌ Puede ser overkill para datos pequeños")

# Ejemplo de early stopping con XGBoost
print("\n🛑 EARLY STOPPING en XGBoost:")
xgb_early = xgb.XGBClassifier(
    n_estimators=1000,  # Muchos estimadores
    max_depth=6,
    learning_rate=0.1,
    early_stopping_rounds=10,  # Para si no mejora en 10 rounds
    random_state=42,
    eval_metric='logloss'
)

# Ajustar con validación
xgb_early.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    verbose=False  # Sin output detallado
)

print(f"   📊 Estimadores usados: {xgb_early.best_iteration}")
print(f"   🎯 De {1000} máximo, paró en {xgb_early.best_iteration}")

Caracteristicas Avanzadas
--------------------------------------------------
🔧 CARACTERÍSTICAS ESPECIALES:

🐢 Gradient Boosting clásico:
   ✅ Fácil de entender
   ✅ Implementación estable
   ❌ Pocos parámetros avanzados
   ❌ No maneja NaN
   ❌ No paralelización eficiente

🚀 XGBoost:
   ✅ Manejo automático de NaN
   ✅ Regularización L1 y L2
   ✅ Early stopping inteligente
   ✅ Paralelización y GPU
   ✅ Multiple importance metrics
   ✅ Cross-validation integrada
   ❌ Más parámetros para tunear
   ❌ Puede ser overkill para datos pequeños

🛑 EARLY STOPPING en XGBoost:
   📊 Estimadores usados: 256
   🎯 De 1000 máximo, paró en 256


## 8) Bonus ejemplo con regresión

In [11]:
#Ejemplo de regresión

print("BONUS: Ejemplo de regresion")
print("-" * 50)

# Datos de regresión
X_reg, y_reg = make_regression(n_samples=1000, n_features=10, noise=0.1, random_state=42)
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.3, random_state=42
)

# Modelos de regresión
gbm_reg = GradientBoostingRegressor(n_estimators=100, random_state=42)
xgb_reg = xgb.XGBRegressor(n_estimators=100, random_state=42)

# Entrenar
gbm_reg.fit(X_train_reg, y_train_reg)
xgb_reg.fit(X_train_reg, y_train_reg)

# Predicciones
gbm_reg_pred = gbm_reg.predict(X_test_reg)
xgb_reg_pred = xgb_reg.predict(X_test_reg)

# Métricas
gbm_mse = mean_squared_error(y_test_reg, gbm_reg_pred)
xgb_mse = mean_squared_error(y_test_reg, xgb_reg_pred)

print("📈 REGRESIÓN - Mean Squared Error:")
print(f"   🐢 Gradient Boosting: {gbm_mse:.4f}")
print(f"   🚀 XGBoost:          {xgb_mse:.4f}")

BONUS: Ejemplo de regresion
--------------------------------------------------
📈 REGRESIÓN - Mean Squared Error:
   🐢 Gradient Boosting: 1321.5928
   🚀 XGBoost:          1854.5063


## 9) Resultados finales

In [12]:
# Resultados finales

print("Resultado Final")
print("-" * 50)

print("🏆 PUNTUACIÓN POR ROUNDS:")
rounds_won = 0

print("\n Round 1 - Manejo de NaN:")
print("   🚀 XGBoost WINS! (manejo automático)")
rounds_won += 1

print("\n Round 2 - Velocidad:")
if xgb_time < gbm_time:
    print("   🚀 XGBoost WINS! (más rápido)")
    rounds_won += 1
else:
    print("   🐢 Gradient Boosting WINS!")

print("\n Round 3 - Precisión:")
if xgb_accuracy > gbm_accuracy:
    print("   🚀 XGBoost WINS! (más preciso)")
    rounds_won += 1
elif gbm_accuracy > xgb_accuracy:
    print("   🐢 Gradient Boosting WINS!")
else:
    print("   🤝 EMPATE!")

print("\n Round 4 - Interpretabilidad:")
print("   🚀 XGBoost WINS! (más opciones)")
rounds_won += 1

print("\n Round 5 - Características:")
print("   🚀 XGBoost WINS! (más features)")
rounds_won += 1

print(f"\n🎉 RESULTADO FINAL:")
print(f"   🚀 XGBoost:          {rounds_won}/5 rounds")
print(f"   🐢 Gradient Boosting: {5-rounds_won}/5 rounds")

if rounds_won >= 3:
    print("\n👑 ¡XGBoost es el CAMPEÓN!")
    print("   🎯 Mejor para: competencias, datos grandes, máxima precisión")
else:
    print("\n👑 ¡Gradient Boosting resiste!")
    print("   🎯 Mejor para: aprendizaje, simplicidad, datasets pequeños")

Resultado Final
--------------------------------------------------
🏆 PUNTUACIÓN POR ROUNDS:

 Round 1 - Manejo de NaN:
   🚀 XGBoost WINS! (manejo automático)

 Round 2 - Velocidad:
   🚀 XGBoost WINS! (más rápido)

 Round 3 - Precisión:
   🐢 Gradient Boosting WINS!

 Round 4 - Interpretabilidad:
   🚀 XGBoost WINS! (más opciones)

 Round 5 - Características:
   🚀 XGBoost WINS! (más features)

🎉 RESULTADO FINAL:
   🚀 XGBoost:          4/5 rounds
   🐢 Gradient Boosting: 1/5 rounds

👑 ¡XGBoost es el CAMPEÓN!
   🎯 Mejor para: competencias, datos grandes, máxima precisión


## 10) Conclusion y Consejos, cuando usar cada uno

* 🐢 Gradient Boosting → Aprender conceptos, prototipos rápidos
* 🚀 XGBoost → Producción, competencias, máximo rendimiento

